#### IBM Quantum Challenge Africa 2021より
# 量子コンピューターで農地収穫量の最適化！

みなさん、「量子コンピューターで農地収穫量の最適化！」へようこそ。本ハンズオンでは、量子計算における問題のモデル化の例として、IBM Quantum Challenge Africa 2021で取り上げられた「農地収量の最適化問題」を解説します。ここで使用しているコードは、[Quantum Challenge Lab1のnotebook](https://github.com/qiskit-community/ibm-quantum-challenge-africa-2021/blob/main/content/lab1/lab1.ipynb)がベースになっています。

**前提知識**：Python

**事前準備**：[IBM Quantum](https://quantum-computing.ibm.com/)へのサインアップ


## 目次
1. [はじめに](#Introduction)
1. [Qiskit](#Qiskit)
1. [2次計画問題](#QuadraticProblem)
1. [農地の収穫量問題](#CropYieldProblem)
1. [Quadratic Unconstrained Binary Optimization (QUBO)](#QUBO)
1. [古典的な解法](#ClassicalSolution)
1. [量子的な解法](#QuantumSolution)
    1. [QAOAによる解法](#QAOASolution)
    1. [VQEによる解法](#VQESolution)
1. [まとめ](#Summary)
1. [参考文献](#Reference)

## はじめに <a id='Introduction'></a>

量子コンピューターは、古典コンピューターでは解けない問題を解決する可能性を秘めています。

実社会で有用な計算において量子コンピューターが古典コンピューターの計算速度を上回り、その優位性を示す領域を特定することは、量子コンピューターが実用化される時代に向けてとても重要なことです。このような優位性が期待される領域には、機械学習、計算化学、組み合わせ最適化問題などが挙げられます。今回は農地の収穫量を題材に、組み合わせ最適化問題を取り上げます。このハンズオンを通じ、量子コンピューターの優位性と今抱える問題について理解することができるでしょう。

まずは、ハンズオンを実行する環境を準備しましょう。

1. Jupyter notebookのダウンロード<br/>
  ハンズオンで使用するJupyter notebookファイルをダウンロードします

2. [IBM Quantum](https://quantum-computing.ibm.com/)へのログイン

3. ステップ1でダウンロードしたファイルのアップロード

  3-1. [Dashbord](https://quantum-computing.ibm.com/)上の[Launch Lab](https://quantum-computing.ibm.com/lab)ボタンをクリックします
  
  3-2. Upload Filesボタン(上矢印)を押して、ステップ1でダウンロードしたファイルをアップロードします <br/>

4. アップロードしたファイル「KQC22_3_crop.ipynb」を開いてください

### 準備：必要なモジュールとライブラリのインポート
次に、必要なモジュールとライブラリをインポートしておきましょう。Jupyter notebookでは、セルにカーソルを置き、Shift+Enterを押すと、セル内のコードが実行されます。

In [1]:
# 補助ライブラリのインポート
import numpy as np

# Qiskitのインポート
from qiskit import IBMQ, Aer
from qiskit.algorithms import QAOA, VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA
from qiskit.providers.ibmq import least_busy
from qiskit.utils import QuantumInstance
from qiskit.tools import job_monitor

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo

以上で準備は完了です。

## 農地の収穫量問題 <a id="CropYieldProblem"></a>
農場の作物や経営を最適化して、リスクを減らしながら利益を上げたいというニーズはよくあります。アフリカや全世界が直面している大きな課題の1つは、すべての人に十分な食料をいかに生産するかということです。今回問題とするのは、利益ではなく、収穫した作物のトン数です。農地の広さが決められ、植えられる作物の種類が与えられたとき、どの作物をそれぞれ何ヘクタール植えたら、収穫量を最大にできるかが課題になります。

<table>
    <tr>
        <th>
            <img src="farm_template.svg" width="384px"/>
        </th>
    </tr>
    <tr>
        <th>
            あなたの経営する美しい3ha(ヘクタール)の農場
        </th>
    </tr>
</table>

<table>
    <tr>
        <th>
        <img src="crop_wheat.svg" width="256px"/>
        </th>
        <th>
            <img src="crop_soybeans.svg" width="256px"/>
        </th>
        <th>
            <img src="crop_maize.svg" width="256px"/>
        </th>
        <th>
            <img src="crop_pushpull.svg" width="256px"/>
        </th>
    </tr>
    <tr>
        <th>
            小麦
        </th>
        <th>
            大豆
        </th>
        <th>
            トウモロコシ
        </th>
        <th>
            プッシュプル
        </th>
<!--         <th>
            <p align="right" style="height:32px;padding-top:10px;">Wheat<img src="wheat.svg" width="32px" style="float:left;margin-top:-10px;margin-right:8px;"/></p>
        </th>
        <th>
            <p style="height:32px;padding-top:10px;">Soybeans<img src="soybeans.svg" width="32px" style="float:left;margin-top:-10px;margin-right:8px;"/></p>
        </th>
        <th>
            <p style="height:32px;padding-top:10px;">Maize<img src="maize.svg" width="32px" style="float:left;margin-top:-10px;margin-right:8px;"/></p>
        </th>
        <th>
            <p style="height:32px;padding-top:10px;">Push-Pull<img src="pushpull.svg" width="32px" style="float:left;margin-top:-10px;margin-right:8px;"/></p>
        </th> -->
    </tr>
</table>
農法には「単作」「間作」「プッシュプル農法」の3種類があります。単作とは、一つの作物だけを栽培する方法です。単作では、病気や害虫の影響を受けやすく、収穫量全体に影響を及ぼします。また、異なる作物を近くで栽培すると、両方の収穫量が増えたり、逆に収穫量が減ったりします。間作とは、収穫量を増やすために2つの異なる作物を選択することです。プッシュプル農法とは、害虫を寄せ付けないプッシュ作物と、害虫を引き寄せるプル作物をペアで栽培することです。これを大規模な農場に組み込むことで、収穫量を増やすことができます。ただ、プッシュプル作物は利用できなかったり食用にならないため、プッシュプル作物を全体の収穫量として使用できません。
<table>
    <tr>
        <th>
        <img src="farm_mono.svg" width="256px"/>
        </th>
        <th>
            <img src="farm_intercrop.svg" width="256px"/>
        </th>
        <th>
            <img src="farm_intercrop_pushpull.svg" width="256px"/>
        </th>
    </tr>
    <tr>
        <th>
            単作
        </th>
        <th>
            間作
        </th>
        <th>
            プッシュプル農法
        </th>
    </tr>
</table>

このような農地の収穫量問題は、変数の特定の組み合わせで解が得られるという点で、組合せ最適化問題です。ここで紹介する問題は古典的に解くことができるほど小さいものですが、より大きな問題になると、最適化すべき組み合わせの数が多くなるため、古典的なコンピュータでは扱いにくくなります。

### 目的：収穫量の最大化
あなたは$3~ha$の農場のオーナーです。植えられる作物は、小麦、大豆、トウモロコシ、プッシュプルの4種類です。プッシュプルは、収穫しても売ることはできませんが、他の作物の収穫量を増やすことができます。それぞれの作物は$0~ha$または$1~ha$作付けすることができます。この農場の収穫量(トン)を以下のような2次関数として定義します。この2次関数の変数は、作付けする作物の作付け面積（ヘクタール数）です。解きたい問題は、どの作物の組合せを選べば、最大の収穫量が得られるか(2次関数を最大化できるか)ということです。

#### 作物の種類
<img src="qubo_problem_graphical_variables.svg" width="534px"/> <br>
$~$$~$$~$$~$$~$$~$$~$$~$$~$$~$$~$$~$$~$$~$$~$$~$小麦　　　　　　　大豆　　　　　　トウモロコシ　　　　　プッシュプル

#### 問題の定式化
<img src="qubo_problem_graphical.svg" width="400px"/>


$$
\begin{align}
    \text{maximize} \quad & 2(\operatorname{Wheat}) + \operatorname{Soybeans} + 4(\operatorname{Maize}) \\
    & + 2.4(\operatorname{Wheat}\times\operatorname{Soybeans}) \\ & + 4(\operatorname{Wheat}\times\operatorname{Maize})\\
    &+ 4(\operatorname{Wheat}\times\operatorname{PushPull}) \\ & + 2(\operatorname{Soybeans}\times\operatorname{Maize}) \\
                          & + (\operatorname{Soybeans}\times\operatorname{PushPull}) \\ & + 5(\operatorname{Maize}\times\operatorname{PushPull})
\end{align}
$$

$$
\begin{align}
\text{subject to} \quad & \operatorname{Wheat} + \operatorname{Soybeans} + \operatorname{Maize} + \operatorname{PushPull} \leq{} 3\\
& 0\leq{}\operatorname{Wheat}\leq{}1\\
& 0\leq{}\operatorname{Soybeans}\leq{}1\\
& 0\leq{}\operatorname{Maize}\leq{}1\\
& 0\leq{}\operatorname{PushPull}\leq{}1
\end{align}
$$

### 定式化からの2次計画問題の作成
上記で定義した問題を表す`QuadraticProgram`を作成してみましょう。以下の `cropyield_quadratic_program` 関数を完成させてください。上記の目的関数にある各変数の係数をみながら、以下の「**#目的関数の定義**」と「**#制約の追加**」の各コメント下の式にそれぞれ係数を代入してみましょう。

変数名は何でもよいですが、ここでは `Wheat`、 `Soybeans`、 `Maize`、 および `PushPull` の頭文字`W`,`S`,`M`,`P`を使っています。

In [3]:
# 穴埋め問題
def cropyield_quadratic_program():
    cropyield = QuadraticProgram(name="Crop Yield")
    ##############################
    # ここにコードを書いてください
    # 変数の追加
    cropyield.binary_var(name="W") #Wheatの変数
    cropyield.binary_var(name="S") #Soybeansの変数
    cropyield.binary_var(name="M") #Maizeの変数
    cropyield.binary_var(name="P") #Pushpullの変数
    
    # 目的関数の定義
    cropyield.maximize(
        linear={"W":2, "S":1, "M":4},
        quadratic={("W", "S"):2.4 , ("W", "M"):4 , ("W", "P"):4 ,("S", "M"):2 , ("S", "P"):1 , ("M", "P"):5 },
    )
    
    # 制約の追加
    cropyield.linear_constraint(linear={"W":1 , "S":1 ,"M": 1, "P":1 }, sense="<=", rhs=3)
    
    
    ##############################
    return cropyield
# 問題をLP文字列で出力
cropyield = cropyield_quadratic_program()
print(cropyield.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Crop Yield

Maximize
 obj: 2 W + S + 4 M + [ 4.800000000000 W*S + 8 W*M + 8 W*P + 4 S*M + 2 S*P
      + 10 M*P ]/2
Subject To
 c0: W + S + M + P <= 3

Bounds
 0 <= W <= 1
 0 <= S <= 1
 0 <= M <= 1
 0 <= P <= 1

Binaries
 W S M P
End



## 量子コンピューターで解くための準備 <br/> QUBO形式への変換 <a id="QUBO"></a>


最適化問題を特定の量子アルゴリズムで解くためには、通常、適性なフォーマットに変換して適用可能な状態にする必要があります。今回のような2次計画問題を量子コンピューター解く際には、Quadratic Unconstrained Binary Optimization (QUBO) 形式という特殊な形式に変換します。

興味深いことに、この農地収穫量最適化問題は、ハミルトニアンの基底状態(一番低いエネルギー状態）を求める問題として解くことができます。ハミルトニアンとは、シミュレーションしたい物理系の全エネルギーを表すエネルギー関数と考えることができます。この物理系は、さらに [**イジングモデル**](https://en.wikipedia.org/wiki/Ising_model) と呼ばれる数学モデルで表現することができます。この数学モデルは、電子のスピンの上向き・下向きを、バイナリ変数の0、1に読み替えることで、QUBO問題をイジングモデルにマップするのです。
    
今回適用するVQEやQAOAのような変分量子アルゴリズムは[**二次非制約二次最適化(QUBO)**](https://en.wikipedia.org/wiki/Quadratic_unconstrained_binary_optimization)という形式に問題を変換する必要があり、 Qiskitはこの変換機能を提供します。

## 演習: 農地収穫量問題のQUBOへの変換
私たちの農地収穫量問題をQUBOに変換しましょう。

In [4]:
# 農地収穫量問題をQUBO形式に変換します。
##############################
# ここにコードを書いてください。

cropyieldQUBO=QuadraticProgramToQubo().convert(cropyield)

#
##############################
print(cropyieldQUBO.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Crop Yield

Minimize
 obj: - 160.400000000000 W - 159.400000000000 S - 162.400000000000 M
      - 158.400000000000 P - 158.400000000000 c0@int_slack@0
      - 316.800000000000 c0@int_slack@1 + [ 52.800000000000 W^2
      + 100.800000000000 W*S + 97.600000000000 W*M + 97.600000000000 W*P
      + 105.600000000000 W*c0@int_slack@0 + 211.200000000000 W*c0@int_slack@1
      + 52.800000000000 S^2 + 101.600000000000 S*M + 103.600000000000 S*P
      + 105.600000000000 S*c0@int_slack@0 + 211.200000000000 S*c0@int_slack@1
      + 52.800000000000 M^2 + 95.600000000000 M*P
      + 105.600000000000 M*c0@int_slack@0 + 211.200000000000 M*c0@int_slack@1
      + 52.800000000000 P^2 + 105.600000000000 P*c0@int_slack@0
      + 211.200000000000 P*c0@int_slack@1 + 52.800000000000 c0@int_slack@0^2
      + 211.200000000000 c0@int_slack@0*c0@int_slack@1
      + 211.200000000000 c0@int_slack@1^2 ]/2 + 237.600000000000
Subject To

Bo

## 古典的な解法 <a id="ClassicalSolution"></a>

以上で、農地収穫量の最適化問題を量子コンピューターで解ける形に定義することができました。この問題はさほど大きくないので、古典的に厳密解を求めることも可能です。実際に量子コンピューターで解く前に、古典的に解を求めておきましょう。

古典的な解は、NumpyとQiskitを使って簡単に得ることができます。QUBO問題を解くことは、基礎となる行列表現の最小固有値を見つけることに他なりません。そして幸いなことに、この行列表現がどのようなものかを知る必要はなく、定義したQUBO問題を、`NumPyMinimumEigensolver`と`MinimumEigenOptimizer`に渡すだけでよいのです。

古典解を求める関数は以下のように定義できます。まずソルバーとして`NumPyMinimumEigensolver`インスタンスを持つ、`MinimumEigenOptimizer` オプティマイザーを作成します。与えられた2次計画問題を引数として`solve` メソッドを呼ぶと、オプティマイザーは，問題をパラメーター化された表現に変換してソルバーに渡し、ソルバーはパラメーターを最適化することで、行列表現の最小固有値を求め、最終的に元の問題の解を得ることができます。

In [5]:
def get_classical_solution_for(quadprog: QuadraticProgram):
    # Create solver
    solver = NumPyMinimumEigensolver()

    # Create optimizer for solver
    optimizer = MinimumEigenOptimizer(solver)

    # Return result from optimizer
    return optimizer.solve(quadprog)

私たちの農地収量問題を解いてみましょう。

In [6]:
# Get classical result
classical_result = get_classical_solution_for(cropyield)

# Format and print result
print("古典の解法で求める参考値\n")
print(f"最大収穫量は {classical_result.fval} トン")
print(f"作付けすべき作物は: ")

_crops = [v.name for v in cropyield.variables]
for cropIndex, cropHectares in enumerate(classical_result.x):
    print(f"\t{_crops[cropIndex]}を{cropHectares}ha")

古典の解法で求める参考値

最大収穫量は 19.0 トン
作付けすべき作物は: 
	Wを1.0ha
	Sを0.0ha
	Mを1.0ha
	Pを1.0ha


## 量子コンピューターを使って解いてみよう <a id="QuantumSolution"></a>

では、量子コンピューターを使用して、農地収穫量問題を解いてみましょう。

### 量子アルゴリズムによる解法 <a id="QAOASolution"></a>
量子ゲート方式による組合せ最適化問題の解を求めるためのアルゴリズムの一つであるQAOAを使って問題を解きます。先ほど定義した関数で、古典的ソルバーを`QAOA`クラスのインスタンスで置き換えるだけです。

In [7]:
def get_QAOA_solution_for(
    quadprog: QuadraticProgram, quantumInstance: QuantumInstance, optimizer=None,
):
    _eval_count = 0

    def callback(eval_count, parameters, mean, std):
        nonlocal _eval_count
        _eval_count = eval_count

    # Create solver
    solver = QAOA(
        optimizer=optimizer, quantum_instance=quantumInstance, callback=callback,
    )

    # Create optimizer for solver
    optimizer = MinimumEigenOptimizer(solver)

    # Get result from optimizer
    result = optimizer.solve(quadprog)
    return result, _eval_count

#### シミュレーターでの実行
量子コンピューターでの実行準備ができました。まずはシミュレーターで実行してみましょう。QiskitのAerコンポーネントの`qasm_simulator`を使用して、`QuantumInstance`を作成し、`get_QAOA_solution_for` 関数に渡します。

In [8]:
# We will use the Aer provided QASM simulator
backend = Aer.get_backend("qasm_simulator")

# Create a QuantumInstance
simulator_instance = QuantumInstance(backend=backend)

# Get QAOA result
qaoa_result, qaoa_eval_count= get_QAOA_solution_for(cropyield, simulator_instance)

# Format and print result
print("量子ハイブリッドアルゴリズムであるQAOAを用いた計算結果:\n")
print(f"最大収穫量は {qaoa_result.fval} トン")
print(f"作付けした作物は:")
for cropHectares, cropName in zip(qaoa_result.x, qaoa_result.variable_names):
    print(f"\t{cropName}を{cropHectares}ha")

量子ハイブリッドアルゴリズムであるQAOAを用いた計算結果:

最大収穫量は 19.0 トン
作付けした作物は:
	Wを1.0ha
	Sを0.0ha
	Mを1.0ha
	Pを1.0ha


#### 実量子ハードウェア上での実行
次に実量子ハードウェアで実行したいところですが、実は私たちはこの量子回路を実量子ハードウェア上で動作させることができません。先ほど見たように私たちの問題を解くためには、6量子ビットが必要です。今私たちが使用できる量子コンピューターの量子ビット数を確認しましょう。

まず、IBM Quantumのアカウントをロードします。次のセルがエラーになる方は、この[クイックガイド](https://quantum-computing.ibm.com/lab/docs/iql/manage/account/ibmq)に従って、アカウントを有効にしてください。

次に、使用できるハードウェアの名前とビット数を確認します。

In [9]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-education', group='ibm-3', project='kawasaki-camp')
IBMQ.providers() 

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>,
 <AccountProvider for IBMQ(hub='ibm-q-internal', group='deployed', project='default')>,
 <AccountProvider for IBMQ(hub='ibm-q-education', group='ibm-2', project='open-house-qiski')>,
 <AccountProvider for IBMQ(hub='ibm-q-education', group='ochanomizu-unive', project='introduction-of-')>,
 <AccountProvider for IBMQ(hub='ibm-q-education', group='ibm-research-tok-1', project='p-tech-quantum')>,
 <AccountProvider for IBMQ(hub='ibm-q-education', group='ibm-3', project='nict-quantum-cam')>,
 <AccountProvider for IBMQ(hub='ibm-q-education', group='ibm-3', project='kawasaki-camp')>,
 <AccountProvider for IBMQ(hub='ibm-q-education', group='ibm-4', project='qiskit-hackathon')>,
 <AccountProvider for IBMQ(hub='strangeworks-hub', group='qc-com', project='runtime')>]

In [10]:
for _backend in IBMQ.get_provider(hub='ibm-q-education', group='ibm-3', project='kawasaki-camp').backends():
    if not (_backend.configuration().simulator):
        print(f"{_backend.name()} : {_backend.configuration().n_qubits} qubit")

ibmq_lima : 5 qubit
ibmq_belem : 5 qubit
ibmq_quito : 5 qubit
ibmq_jakarta : 7 qubit
ibmq_manila : 5 qubit
ibm_lagos : 7 qubit
ibm_nairobi : 7 qubit
ibm_perth : 7 qubit
ibm_oslo : 7 qubit


In [11]:
# #一番空いているバックエンドを自動的に選択して実行
real_backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 6 
                             and not x.configuration().simulator))
print("最も空いているバックエンドは: ", real_backend)

最も空いているバックエンドは:  ibm_perth


In [ ]:
# 実機での計算結果を求めます。
quantum_instance_real = QuantumInstance(real_backend, shots=2048)

# オプティマイザーを指定します。
optimizer = COBYLA(maxiter=50)

## QAOAという量子アルゴリズムをつかって実機上での計算を行います。
qaoa_result_real, qaoa_eval_count_real = get_QAOA_solution_for(
    cropyield, quantum_instance_real, optimizer=optimizer
)
# Format and print result
print("量子アルゴリズムをつかった計算結果:\n")
print(f"最大収穫量は {qaoa_result_real.fval} トン")
print(f"作付けした作物は: ")
for cropHectares, cropName in zip(qaoa_result_real.x, qaoa_result_real.variable_names):
    print(f"\t{cropName}を{cropHectares}ha")

print(f"\nThe solution was found within {qaoa_eval_count} evaluations of QAOA.")

### (オプション) VQEによる解法 <a id="VQESolution"></a>

QUBO問題を解くには、`VQE` アルゴリズムを使用することもできます。使用方法は以下の通りです。`VQE` のインスタンスも反復処理を行いますので、作物収量問題の解決策を見つけるために何回の反復処理が必要かを測定することができます。

In [ ]:
def get_VQE_solution_for(
    quadprog: QuadraticProgram, quantumInstance: QuantumInstance, optimizer=None,
):
    _eval_count = 0

    def callback(eval_count, parameters, mean, std):
        nonlocal _eval_count
        _eval_count = eval_count

    # Create solver and optimizer
    solver = VQE(
        optimizer=optimizer, quantum_instance=quantumInstance, callback=callback
    )

    # Create optimizer for solver
    optimizer = MinimumEigenOptimizer(solver)

    # Get result from optimizer
    result = optimizer.solve(quadprog)
    return result, _eval_count

シミュレーターに対して実行してみてください。以前とほぼ同じ答えが返ってくるはずです(乱数のシードを固定していないので多少の変動があります)。

In [ ]:
# We will use the Aer provided QASM simulator
backend = Aer.get_backend("qasm_simulator")

# Create a QuantumInstance
simulator_instance = QuantumInstance(backend=backend)

# Get VQE result
vqe_result, vqe_eval_count = get_VQE_solution_for(cropyield, simulator_instance)

# Format and print result
print("Solution found using the VQE method:\n")
print(f"Maximum crop-yield is {vqe_result.fval} tons")
print(f"Crops used are: ")
for cropHectares, cropName in zip(vqe_result.x, vqe_result.variable_names):
    print(f"\t{cropHectares} ha of {cropName}")

print(f"\nThe solution was found within {vqe_eval_count} evaluations of VQE")

## まとめ <a id="Summary"></a>

いかがでしたか？本日はハンズオンを通して以下を説明しました：

- 最適化問題を解くための流れ
- 定式化からの2次計画問題作成方法（係数の入力）
- 2次計画問題のQUBO形式への変換
- 最適化問題を分子の基底エネルギーを求める問題に帰着できる
- 量子アルゴリズムを適用した解法

量子コンピューターが最適化問題の解法として優越性が期待されているものの、実際の問題を解くためには、量子ビット数を削減できるようなモデルに落とし込むことが重要です。

この問題が取り上げられたQuantum Challenge 2021 Africaの問題は[こちら](https://github.com/qiskit-community/ibm-quantum-challenge-africa-2021)から取得可能です。このハンズオンでご紹介した問題のほか、金融や創薬の問題があります。また先日終了した[Quantum Challenge 2021 Fall](https://github.com/qiskit-community/ibm-quantum-challenge-fall-2021)にも興味深い問題がたくさんあります。こちらは日本語訳もありますので、より取り組みやすいと思います。来年もChallengeがあると予想されます。規定数の問題を解くとバッジが取得できますので、みなさまもぜひ挑戦してみてください！

また、Qiskitをもっと知りたい方には、Qiskit Documentationの[Tutorial](https://qiskit.org/documentation/tutorials.html)を、量子コンピューターについてもっと勉強したい方には、[Qiskit Textbook](https://qiskit.org/textbook/preface.html)をお勧めします。TutorialはIBM Quantumの[Quantum Lab](https://quantum-computing.ibm.com/lab)からも参照＆実行できます。

## 参考文献 <a id="Reference"></a>
[1] A. A. Nel, ‘Crop rotation in the summer rainfall area of South Africa’, South African Journal of Plant and Soil, vol. 22, no. 4, pp. 274–278, Jan. 2005, doi: 10.1080/02571862.2005.10634721.

[2] H. Ritchie and M. Roser, ‘Crop yields’, Our World in Data, 2013, [Online]. Available: https://ourworldindata.org/crop-yields.

[3] G. Brion, ‘Controlling Pests with Plants: The power of intercropping’, UVM Food Feed, Jan. 09, 2014. https://learn.uvm.edu/foodsystemsblog/2014/01/09/controlling-pests-with-plants-the-power-of-intercropping/ (accessed Feb. 15, 2021).

[4] N. O. Ogot, J. O. Pittchar, C. A. O. Midega, and Z. R. Khan, ‘Attributes of push-pull technology in enhancing food and nutrition security’, African Journal of Agriculture and Food Security, vol. 6, pp. 229–242, Mar. 2018.

In [ ]:
import qiskit.tools.jupyter

#%qiskit_version_table
%qiskit_copyright